# RegressPairFit
This module implements the adjustment based on fitting 2 linear regression models.

In [1]:
from set_up import *
pd.options.display.max_rows=7
import matplotlib.pyplot as plt
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

# Specific for this module
from pybreaks.adjust_linear_model_pair_fitting import RegressPairFit

In [2]:
# this cell loads either test data with properties as set by the user or real data and allows to introduce a break
def load_data(real=False, trend=0.01, bias=0.1, breaktime=datetime(2012,7,1), 
              size=(0.2, 1.1), resample=False):
    if real:
        CAN, REF = load_real_data(707393, breaktime, size, resample)
    else:
        CAN, REF = load_test_data(trend, bias, breaktime, size, resample, seed=1234)
    return CAN, REF

In [3]:
interpol_methods = ['linear', 'poly2', 'poly3', 'nearest', 'zero', 'slinear', 
                    'quadratic', 'cubic', 'previous', 'next']

@interact(TREND=(0, 0.05, 0.01), BIAS=(-0.5, 0.5, 0.1), BREAK_ADD=(-0.4, 0.4, 0.05), 
          BREAK_MULT=(0.7, 1.3, 0.05), RESAMPLE=['NONE', 'INPUT', 'OUTPUT'],
          INTERPOLATION_METHOD=interpol_methods)

def LMP(USE_REAL_DATA=False, TESTDATA_TREND=0.01, TESTDATA_BIAS=0.1, BREAK_ADD=0.2, 
        BREAK_MULT=1.1, RESAMPLE_CORRECTIONS=True, INTERPOLATION_METHOD='linear', 
        RESAMPLE='NONE'):
    
    BREAKTIME=datetime(2012,7,1)
    size=(BREAK_ADD, BREAK_MULT)
    
    resample = True if RESAMPLE=='INPUT' else False
    CAN, REF = load_data(USE_REAL_DATA, TESTDATA_TREND, TESTDATA_BIAS, BREAKTIME, size, 
                         resample)
    
    lmp = RegressPairFit(CAN, REF, BREAKTIME, candidate_freq='M' if resample else 'D',
                         regress_resample=None, bias_corr_method='cdf_match',
                         filter=('both', 5), adjust_group=0, model_intercept=True)
    
    can_adjusted = lmp.adjust(CAN.loc[:BREAKTIME], True, 'M' if resample else 'D',
                              RESAMPLE_CORRECTIONS, INTERPOLATION_METHOD)
    
    ##plotting##
    fig, (ax1) = plt.subplots(1,1, figsize=(15,4)) 
    if RESAMPLE == 'OUTPUT':
        lmp.df_original.resample('M').mean().plot(ax=ax1)
    else:
        lmp.df_original.plot(ax=ax1)
    
    fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(15,4))
    lmp.plot_models(axs=[ax1, ax2])
    
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(15,4))
    lmp.plot_adjustments(ax=ax2)
    
   

aW50ZXJhY3RpdmUoY2hpbGRyZW49KENoZWNrYm94KHZhbHVlPUZhbHNlLCBkZXNjcmlwdGlvbj11J1VTRV9SRUFMX0RBVEEnKSwgRmxvYXRTbGlkZXIodmFsdWU9MC4wMSwgZGVzY3JpcHRpb27igKY=
